# TensorFlow 2 quickstart for experts

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install the TensorFlow 2 package:

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


Import TensorFlow into your program:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import random

import h5py
import tensorflow as tf
import numpy as np
from tensorflow.python.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Input, Reshape, Permute, Lambda, TimeDistributed, SpatialDropout1D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.python.keras.layers.merge import Add, Multiply, multiply
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.utils import np_utils

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [0]:
def load_seq_lst(lst_name, num_seq, ovr_num=None, num_joints=25, num_class=52):
    # the reverse process of load_sequence() method in pku_dataset class
    # the output is a list of skeleton sequence of variable length
    # for training set, samples clips around intervals of actions
    assert (os.path.isfile(lst_name + '.txt') and os.path.isfile(lst_name + '.h5'))
    if ovr_num is None:
        ovr_num = int(num_seq / 2)
    keyname_lst = [item.strip().split() for item in open(lst_name + '.txt', 'r').readlines()]
    X = []
    Y = []
    vid_list = []
    start_list = []
    with h5py.File(lst_name + '.h5', 'r') as hf:
        for item in keyname_lst:
            skeleton = np.asarray(hf.get(item[0]))
            # skeleton = skeleton.reshape((skeleton.shape[0], self._num_joints, self._dim_point ))
            skeleton1 = skeleton[:, 0:75].reshape((skeleton.shape[0], num_joints, 3))
            skeleton2 = skeleton[:, 75:].reshape((skeleton.shape[0], num_joints, 3))
            skeleton = np.concatenate((skeleton1, skeleton2), axis=-1)

            labels = np.asarray(hf.get(item[1]), dtype=int)
            labels_pertime = np.zeros((skeleton.shape[0]), dtype=np.int32)
            for clip_idx in range(len(labels)):
                # notice: for detection, labels start from 1, as there are empty clips for the input stream
                labels_pertime[labels[clip_idx][1]:labels[clip_idx][2]] = labels[clip_idx][0] + 1
            labels_pertime = labels_pertime.astype(np.int32)
            labels_pertime = np_utils.to_categorical(labels_pertime, num_class)

            if skeleton.shape[0] > num_seq:
                start = 0
                while start + num_seq < skeleton.shape[0]:
                    X.append(skeleton[start:start + num_seq])
                    Y.append(labels_pertime[start + num_seq])
                    vid_list.append(item[0])
                    start_list.append(start)
                    start = start + ovr_num
                X.append(skeleton[-num_seq:])
                Y.append(labels_pertime[-1])
                vid_list.append(item[0])
                start_list.append(skeleton.shape[0] - num_seq)
            else:
                skeleton = np.concatenate((np.zeros((num_seq - skeleton.shape[0], skeleton.shape[1], skeleton.shape[2])), skeleton), axis=0)
                labels_pertime = np.concatenate((np.zeros((num_seq - labels_pertime.shape[0], labels_pertime.shape[1])), labels_pertime), axis=0)
                X.append(skeleton)
                Y.append(labels_pertime[-1])
                vid_list.append(item[0])
                start_list.append(0)

    X = np.asarray(X).astype(np.float32)
    Y = np.asarray(Y)
    print(X.shape, Y.shape)
    return X, Y, vid_list, start_list

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
param = {'max_iter': 500, 'step_inter': 80, 'base_learn_rate': 0.01, 'lr_gamma': 0.5, 'weight_regular': 0, 'num_seq': 200, 'batchsize': 1024, 'write_file': True,
         'num_joints': 25, 'num_class': 51 + 1, 'dim_point': 3 * 2, 'key_seq': 30, 'sub_mean': True}
param['trn_file'] = F"/content/drive/My Drive/crs_view_trn_skt"
param['val_file'] = F"/content/drive/My Drive/crs_view_val_skt"
param['save_path'] = F"/content/drive/My Drive/gcn/gcn/weights.hdf5"
param['write_file_name'] = F"/content/drive/My Drive/gcn/gcn.txt"
# no batch size, as one batch only

trainX, trainY, train_vid_list, train_start_list = load_seq_lst(param['trn_file'], param['num_seq'])
valX, valY, val_vid_list, val_start_list = load_seq_lst(param['val_file'], param['num_seq'])

Use `tf.data` to batch and shuffle the dataset:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((valX, valY)).batch(32)

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing):

In [0]:
class DRLModel(Model):    

    def call(self, inputs, mask=None):
        skt_input = Input(shape=(param['num_seq'], param['num_joints'], param['dim_point']))
        data = skt_input

        if param['sub_mean']:
            data = Permute((1, 3, 2))(data)
            data2 = TimeDistributed(TimeDistributed(Dense(1, trainable=False)))(data)
            data2 = Lambda(lambda x: tf.keras.backend.repeat_elements(x, param['num_joints'], axis=-1), output_shape=lambda s: (s[0], s[1], s[2], s[3] * self.param['num_joints']))(data2)
            data = Add()([data, data2])

        data = Reshape((param['num_seq'], param['num_joints'] * param['dim_point']))(data)

        hid_size = 256
        out = Bidirectional(LSTM(hid_size, return_sequences=True))(data)
        # out = SpatialDropout1D(0.05)(out)
        out = Bidirectional(LSTM(hid_size, return_sequences=False))(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(Dropout(0.5)(out))
        # out = Add()([out, mask])
        return Dense(3, activation='softmax')(out)

# Create an instance of the model
model = DRLModel()

Choose an optimizer and loss function for training: 

In [0]:
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.Mean(name='test_accuracy')

In [0]:
def random_select_frame():
    selected_frames = np.full((param['num_seq']), 0.0)
    selected = random.sample(range(0, param['num_seq']), param['key_seq'])
    for s in selected:
        selected_frames[s] = 1
    return selected, selected_frames

In [0]:
def choose_action(actions, selected):
  selected = np.array(selected)
  print("choose action")
  print(selected.shape)
  print(selected)
  print(actions.shape)
  print(actions.numpy())
  for x in range(actions.shape[0] -1):
    print(x)
    print(actions[x])
    if actions[x] == 1:
      if selected[x] != param['key_seq']:
          selected[x] = selected[x] + 1
    elif actions[x] == 2:
      if selected[x] != 0:
          selected[x] = selected[x] - 1

  selected_frames = np.full((param['key_seq']), 0)
  for s in selected:
      selected_frames[s] = 1
  return selected, selected_frames

In [0]:
def evaluate(selected_frames, label):
    # TODO buralar yapılacak
    return random.randint(50, 90) 

In [0]:
def compute_reward(selected_frames, label, previous_accuracy):
    accuracy = evaluate(selected_frames, label)
    reward = 0
    if previous_accuracy is None:
        reward = 0.1
    elif previous_accuracy < accuracy:
        reward = 1
    elif previous_accuracy > accuracy:
        reward = -1
    return reward, accuracy

Use `tf.GradientTape` to train the model:

In [0]:
@tf.function
def train_step(sequence, label):
    selected, selected_frames = random_select_frame()
    total_reward = 0
    for x in range(param['key_seq']):
        with tf.GradientTape() as tape:
            actions = model(sequence, selected_frames)

        selected, selected_frames = choose_action(actions, selected)
        reward, previous_accuracy = compute_reward(selected_frames, label, previous_accuracy)
        total_reward += reward
    gradients = tape.gradient(total_reward * -1, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_accuracy(previous_accuracy)

Test the model:

In [0]:
@tf.function
def test_step(sequence, label):
    selected, selected_frames = random_select_frame()
    total_reward = 0
    for x in range(param['key_seq']):
        with tf.GradientTape() as tape:
            actions = model(sequence, selected_frames)

        selected, selected_frames = choose_action(actions, selected)
        reward, previous_accuracy = compute_reward(selected_frames, label, previous_accuracy)
        total_reward += reward
    gradients = tape.gradient(total_reward * -1, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    test_accuracy(previous_accuracy)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  
  for videos, labels in train_ds:
    for i in range(videos.shape[0]):
      train_step(videos[i], labels[i])

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials).